In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./Raw Data Centraline_20220504_2035.csv')

In [3]:
df.head()

,Unnamed: 0,Name,Adress,SA,Date,Rent
0,551,Victory Court・Lower Floor・Flat D,Tsuen Wan Town Centre,391 ft²,2022-04-30,"12,500"
1,552,Parker33・Upper Floor・Flat B,Sai Wan Ho,210 ft²,2022-04-30,"10,800"
2,553,The Long Beach・Tower 6・Upper Floor・Flat A,Olympic Station,548 ft²,2022-04-30,"21,500"
3,554,Lohas Park・Phase 9B Grand Marini・Tower 2 (2A)・...,Lohas Park,807 ft²,2022-04-30,"26,800"
4,555,Lohas Park・Phase 7A Montara・Tower 2 (2A)・Upper...,Lohas Park,775 ft²,2022-04-30,"21,000"


In [4]:
df.dtypes

Unnamed: 0     int64
Name          object
Adress        object
SA            object
Date          object
Rent          object
dtype: object

In [5]:
#drop NA columns and rows
df = df.drop(columns=['Unnamed: 0'], axis=1)
df = df.dropna(axis=0)
#rename columns
df = df.rename(columns={"Rent": "Price(HK$)", "SA": "SA (ft2)","Adress": "District"})

In [6]:
#convert date to datetime format
df["NDate"] = pd.to_datetime(df["Date"],dayfirst=True,exact=False)
df = df.drop(columns=['Date'], axis=1)
df = df.rename(columns={"NDate": "Date"})

In [7]:
#add a new floor column
def floor_change(x):
    if "Lower" in x:
        return "L"
    elif "Middle" in x:
        return "M"
    elif "Upper" in x:
        return "H"
    else:
        return "Others"
    
df["Floor"] = df["Name"].apply(lambda x: floor_change(x))

In [8]:
#convect SA (ft2), Price(HK$) to int
for i in ['SA (ft2)', 'Price(HK$)']:
    df[i] = df[i].str.replace(r'\D', '')
    df[i] = df[i].astype(int)

#add a new column of unit rent
df["Price/ft2(SA)"] = round(df["Price(HK$)"]/df["SA (ft2)"],2)

C:\Users\HiuC Lau\AppData\Local\Temp\ipykernel_19596\1230595599.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace(r'\D', '')


In [9]:
index_names = df[(df['Floor'] == "")|(df['Date'] >= "2022-05-01")].index
# drop these row indexes
# from dataFrame
df.drop(index_names, inplace = True)

In [10]:
def estate(x):
    try: result = re.search(r'(\w+\s)*\w+\s*・',x).group().replace('・','')
    except: result = x
    return result
df['Estate'] = df['Name'].apply(estate)

In [11]:
import requests
import json

In [12]:
#Find district from estate
headers ={"Accept": "application/json", "Accept-Language" : "en"}
url = "https://www.als.ogcio.gov.hk/lookup"

districts =[]

for i in df['Estate']:
    address = i
    params = {"q":address}
    res = requests.get(url, params=params, headers=headers)
    d = json.loads(res.text)
    try: dist = d['SuggestedAddress'][0]['Address']['PremisesAddress']['EngPremisesAddress']['EngDistrict']['DcDistrict']
    except: dist = 'None'
    districts.append(dist)

In [13]:
df['District1'] = districts

In [14]:
index_nodis = df[df['District1'] == 'None'].index

In [15]:
#find district by address instead of estate
headers ={"Accept": "application/json", "Accept-Language" : "en"}
url = "https://www.als.ogcio.gov.hk/lookup"

for i in index_nodis:
    address = df.at[i,'District']
    params = {"q":address}
    res = requests.get(url, params=params, headers=headers)
    d = json.loads(res.text)
    try: dist = d['SuggestedAddress'][0]['Address']['PremisesAddress']['EngPremisesAddress']['EngDistrict']['DcDistrict']
    except: dist = 'None'
    
    df.at[i,'District1'] = dist

In [16]:
df['District1'] = df['District1'].str.replace(' DISTRICT','')

In [17]:
df = df.drop(columns=['Name','Estate'], axis=1)
df

,District,SA (ft2),Price(HK$),Date,Floor,Price/ft2(SA),District1
0,Tsuen Wan Town Centre,391,12500,2022-04-30,L,31.97,KWAI TSING
1,Sai Wan Ho,210,10800,2022-04-30,H,51.43,EASTERN
2,Olympic Station,548,21500,2022-04-30,H,39.23,YAU TSIM MONG
3,Lohas Park,807,26800,2022-04-30,M,33.21,SAI KUNG
4,Lohas Park,775,21000,2022-04-30,H,27.10,SAI KUNG
...,...,...,...,...,...,...,...
9436,Pak Shek Kok | Tai Po Mid-levels,915,23000,2021-10-05,Others,25.14,TAI PO
9437,Luk Yeung,445,14000,2021-10-05,L,31.46,TSUEN WAN
9438,Nam Cheong Station,379,19500,2021-10-05,M,51.45,SHAM SHUI PO
9439,Lohas Park,793,23000,2021-10-05,L,29.00,SAI KUNG


In [18]:
df.to_csv("cleaned_Raw Data Centraline_20220504_2035(+18dis).csv", index = False)

In [19]:
df

,District,SA (ft2),Price(HK$),Date,Floor,Price/ft2(SA),District1
0,Tsuen Wan Town Centre,391,12500,2022-04-30,L,31.97,KWAI TSING
1,Sai Wan Ho,210,10800,2022-04-30,H,51.43,EASTERN
2,Olympic Station,548,21500,2022-04-30,H,39.23,YAU TSIM MONG
3,Lohas Park,807,26800,2022-04-30,M,33.21,SAI KUNG
4,Lohas Park,775,21000,2022-04-30,H,27.10,SAI KUNG
...,...,...,...,...,...,...,...
9436,Pak Shek Kok | Tai Po Mid-levels,915,23000,2021-10-05,Others,25.14,TAI PO
9437,Luk Yeung,445,14000,2021-10-05,L,31.46,TSUEN WAN
9438,Nam Cheong Station,379,19500,2021-10-05,M,51.45,SHAM SHUI PO
9439,Lohas Park,793,23000,2021-10-05,L,29.00,SAI KUNG


In [20]:
df1 = pd.read_csv("cleaned_cnp 3-yrs_web-scrap (+18dis).csv")

In [21]:
pd.concat([df,df1]).reset_index().drop('index',axis=1)

,District,SA (ft2),Price(HK$),Date,Floor,Price/ft2(SA),District1
0,Tsuen Wan Town Centre,391,12500,2022-04-30 00:00:00,L,31.97,KWAI TSING
1,Sai Wan Ho,210,10800,2022-04-30 00:00:00,H,51.43,EASTERN
2,Olympic Station,548,21500,2022-04-30 00:00:00,H,39.23,YAU TSIM MONG
3,Lohas Park,807,26800,2022-04-30 00:00:00,M,33.21,SAI KUNG
4,Lohas Park,775,21000,2022-04-30 00:00:00,H,27.10,SAI KUNG
...,...,...,...,...,...,...,...
9906,Yau Tong,687,22500,2021-10-06,M,32.75,KWUN TONG
9907,Yau Tong,517,16300,2021-10-06,M,31.53,KWUN TONG
9908,Tseung Kwan O,364,15000,2021-10-05,H,41.21,SAI KUNG
9909,Tseung Kwan O,484,14500,2021-10-05,L,29.96,SAI KUNG


In [22]:
df.to_csv("fulldata(+18dis).csv", index = False)